In [118]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
import requests
import nltk
from bs4 import BeautifulSoup
%matplotlib inline

In [14]:
# get fund code
url = "http://www.yesfund.com.tw/w/wq/wq01.djhtm?fbclid=IwAR3x-JBkHxHawdA3dv3-PmFMtR-FP3LxT5nk1S-tvPaR3q9Zgz2jZbYdOwE"
response = requests.get(url).text
soup = BeautifulSoup(response, 'html.parser')

In [86]:
a_list = soup.find_all("a")
code_name_tup = []
code_name_tup_full = []
code_name_dict = {}
for a in a_list:
    code = a["href"].split("_")[1].split(".")[0].split("-")[0]
    name = a.text.split(" ")[-1]
    code_name_tup_full.append((code,name))
    if "累積型" in name:
        name = name.replace("累積型","累積類型")
    if "月配型" in name:
        name = name.replace("月配型","月配類型")
    if "季配型" in name:
        name = name.replace("季配型","季配類型")
    if "(台幣)" in name:
        name = name.replace("(台幣)","新臺幣計價")
    if "(人民幣)" in name:
        name = name.replace("(人民幣)","人民幣計價")
    name = name.split("(")[0]
    code_name_tup.append((code,name))
    try:
        code_name_dict[name] = code
    except:
        print(name)

In [87]:
code_name_tup_full

[('BFZICA', ''),
 ('BFZICA', ''),
 ('BFZICA', ''),
 ('BFZICA', ''),
 ('BFZICA', ''),
 ('BFZICA', ''),
 ('BFZICA', ''),
 ('ACKH44', '野村全球不動產證券化基金-累積型(人民幣)(本基金之配息來源可能為本金)'),
 ('ET008001', '不動產證券化型'),
 ('ACKH43', '野村全球不動產證券化基金-月配型(人民幣)(本基金之配息來源可能為本金)'),
 ('ET008001', '不動產證券化型'),
 ('ACKH53', '野村動態配置多重資產基金-累積型(澳幣)(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金)'),
 ('ET004005', '多重資產型'),
 ('ACKH54', '野村動態配置多重資產基金-月配型(澳幣)(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金)'),
 ('ET004005', '多重資產型'),
 ('ACKH126', '野村六年目標到期新興市場債券基金(台幣)(本基金有相當比重投資於非投資等級之高風險債券)'),
 ('ET005008', '債券型海外債券全球新興市場'),
 ('ACKH116', '野村全球金融收益基金-月配型(台幣)(本基金配息來源可能為本金)'),
 ('ET005003', '債券型海外債券投資等級'),
 ('ACKH113', '野村全球金融收益基金-累積型(台幣)(本基金配息來源可能為本金)'),
 ('ET005003', '債券型海外債券投資等級'),
 ('ACKH96', '野村四年階梯到期新興市場債券基金-季配型(人民幣)(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金)'),
 ('ET005003', '債券型海外債券投資等級'),
 ('ACKH93', '野村四年階梯到期新興市場債券基金-累積型(人民幣)(本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金)'),
 ('ET005003', '債券型海外債券投資等級'),
 ('ACIC15', '野村全球不動產證券化基金-累積型(台幣)(本基金之配息來源可能為本金)'),
 ('ET008001', '

In [2]:
invest = pd.read_excel("invest_data.xlsx")

In [55]:
pd.DataFrame(invest["基金簡稱"].unique()).to_csv("fund_name_code.csv", encoding="utf_8_sig", index=None)

In [77]:
code = []
invest_name = [n.replace(" ","") for n in invest["基金簡稱"].unique()]

In [81]:
code = []
for name in invest_name:
    if name in code_name_dict:
        code.append(code_name_dict[name])
    else:
        code.append('NaN')

In [84]:
pd.DataFrame(data={"name":invest["基金簡稱"].unique(),"code":code}).to_csv("fund_name_code.csv", encoding="utf_8_sig", index=None)

In [ ]:
https://docs.google.com/spreadsheets/d/1K4VwQfRFwBDVnkHF7fEIGjDRfxegreY_9np0UwSG29k/edit?usp=sharing

# 計算 標準差 和 sharp ratio


In [119]:
fund_name_code = pd.read_excel("fund_name_code.xlsx")

In [120]:
fund_name_code.head()

,name,code,sharp_ratio,std
0,野村優質基金-累積類型新臺幣計價,ACIC01,0.493156,0.167101
1,野村 e科技基金,ACIC06,0.414397,0.213106
2,野村中小基金-累積類型,ACIC08,0.494425,0.170284
3,野村全球高股息基金-累積型新臺幣計價,ACIC10,0.428375,0.112011
4,野村精選貨幣市場基金,ACKH22,18.697616,0.000149


In [147]:
len(price)

183

[9.9321,
 9.936,
 9.9393,
 9.9402,
 9.9406,
 9.9422,
 9.9498,
 9.9526,
 9.953,
 9.9567,
 9.9577,
 9.9587,
 9.9599,
 9.9602,
 9.9603,
 9.964,
 9.9671,
 9.9676]

In [150]:
# 爬取基金淨值
sharp_list = []
std_list = []
ES_list = []

for fund in fund_name_code["code"]:
    api = "https://www.moneydj.com/funddj/bcd/tBCDNavList.djbcd?a={}&B=2013-5-24&C=2019-5-24".format(fund)
    response = requests.get(api)
    content = response.text
    try:
        date = content.split(" ")[0].split(",")
        price = np.array([float(num) for num in content.split(" ")[1].split(",")])
        price_prev = np.roll(price,1)
        returns = ((price-price_prev)/price_prev)[1:]
        # 夏普率= [(每日報酬率平均值- 無風險利率) / (每日報酬的標準差)]x (252平方根)
        rf = 0.011/252
#         rf = 0
        mean = np.mean(returns)
        std = np.std(returns)
        sharp_ratio = ((mean-rf)/std)*252**0.5
        sharp_list.append(sharp_ratio)
        std_list.append(std*252**0.5)
        
        # 周資料
        price_prev = np.roll(price,7)
        weekly_returns = ((price-price_prev)/price_prev)[7:]
        ES = np.mean(sorted(weekly_returns)[:int(len(weekly_returns)*0.1)])
        ES_list.append(ES)
    except:
        sharp_list.append(0)
        std_list.append(0)
        ES_list.append(0)
        print(fund)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [152]:
fund_name_code["sharp_ratio"] = sharp_list
fund_name_code["std"] = std_list
fund_name_code["ES"] = ES_list

In [153]:
fund_name_code

,name,code,sharp_ratio,std,ES
0,野村優質基金-累積類型新臺幣計價,ACIC01,0.420179,0.167070,-0.051237
1,野村 e科技基金,ACIC06,0.353621,0.213087,-0.068241
2,野村中小基金-累積類型,ACIC08,0.421352,0.170263,-0.052389
3,野村全球高股息基金-累積型新臺幣計價,ACIC10,0.317579,0.112086,-0.034467
4,野村精選貨幣市場基金,ACKH22,-55.294615,0.000149,0.000027
5,野村全球生技醫療基金,ACIC19,0.410407,0.169253,-0.052038
6,野村新興傘型之大俄羅斯基金,NaN,0.000000,0.000000,0.000000
7,野村中國機會基金,ACIC26,0.311141,0.179978,-0.053455
8,野村全球美元投資級公司債基金-累積型,NaN,0.000000,0.000000,0.000000
9,野村巴西證券投資信託基金,NaN,0.000000,0.000000,0.000000


In [154]:
fund_name_code.to_excel("fund_name_risk.xlsx", index=None,)